In [1]:
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN, AgglomerativeClustering

c:\Users\khans\miniconda3\envs\managedata\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

modules.json: 100%|██████████| 349/349 [00:00<?, ?B/s] 
c:\Users\khans\miniconda3\envs\managedata\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\khans\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config_sentence_transformer

In [4]:
# Read in text file that shows PROSE program to transformation
file_path = '../data/programs_one_to_one.txt'

with open(file_path, 'r') as file:
    data = file.read()

# Split the data on the word "Program:" but keep it as part of the split results
programs_transformations = data.split("\n")
programs_transformations.remove("")

programs = []
transformations = []
lhs = []
rhs = []

for item in programs_transformations:
    item_split = item.split("==>")
    transformation = item_split[1]
    programs.append(item_split[0])
    transformations.append(transformation)
    transformation_split = transformation.split("->")
    lhs.append(transformation_split[0])
    rhs.append(transformation_split[1])



df = pd.DataFrame({
    'program': programs,
    'transformation': transformations,
    'lhs': lhs,
    'rhs': rhs
})

df

,program,transformation,lhs,rhs
0,"let columnName = ""0"" in let x = ChooseInput(vs...",Computers as Theatre->Computers As Theatre,Computers as Theatre,Computers As Theatre
1,"Concat(let columnName = ""0"" in let x = ChooseI...",Computers As Theatre->Computers as Theatre,Computers As Theatre,Computers as Theatre
2,"Concat(ConstStr(""\""""), Concat(let columnName =...","Brenda Laurel->""Laurel, Brenda""",Brenda Laurel,"""Laurel, Brenda"""
3,"Concat(let columnName = ""0"" in let x = ChooseI...","""Laurel, Brenda""->Brenda Laurel","""Laurel, Brenda""",Brenda Laurel
4,"Concat(let columnName = ""0"" in let x = ChooseI...",Tru64 Unix Troubleshooting Diagnosing &amp;->T...,Tru64 Unix Troubleshooting Diagnosing &amp;,Tru64 UNIX Troubleshooting : Diagnosing and Co...
...,...,...,...,...
485,"Concat(ConstStr(""\""""), Concat(let columnName =...","""Farin, Gerald""->""Gerald E. Farin,""","""Farin, Gerald""","""Gerald E. Farin,"""
486,"Concat(let columnName = ""0"" in let x = ChooseI...",Computer Systems Design and Architecture->Comp...,Computer Systems Design and Architecture,Computer Systems Design and Architecture (2nd ...
487,"let columnName = ""0"" in let x = ChooseInput(vs...",Computer Systems Design and Architecture (2nd ...,Computer Systems Design and Architecture (2nd ...,Computer Systems Design and Architecture
488,"Concat(ConstStr(""\""Vincent P. ""), Concat(let c...","Heuring->""Vincent P. Heuring, Harry F. Jordan""",Heuring,"""Vincent P. Heuring, Harry F. Jordan"""


In [5]:
file_path = '../data/transformation_feature.tsv'

df.to_csv(file_path, sep='\t', index=False)


In [6]:
sample = pd.read_csv("../data/SampleData.csv", header=None)
source = sample.iloc[:, 0]
print(source)
transformation_embeddings = [model.encode(text) for text in source]
cosine_similarities = cosine_similarity(transformation_embeddings)
print(cosine_similarities)

0    Concat(ConstStr("\""), Concat(let columnName =...
1    Concat(let columnName = "0" in let x = ChooseI...
2    Concat(let columnName = "0" in let x = ChooseI...
3    Concat(ConstStr("\""), Concat(let columnName =...
4    let columnName = "0" in let x = ChooseInput(vs...
5    Concat(ConstStr("\""), Concat(let columnName =...
Name: 0, dtype: object
[[0.99999994 0.9542013  0.9236096  0.98526764 0.82243896 0.76877   ]
 [0.9542013  1.0000002  0.95138085 0.94090676 0.9172367  0.7299444 ]
 [0.9236096  0.95138085 1.0000002  0.95094514 0.87627673 0.726096  ]
 [0.98526764 0.94090676 0.95094514 1.0000002  0.81625986 0.7591712 ]
 [0.82243896 0.9172367  0.87627673 0.81625986 0.99999976 0.69265276]
 [0.76877    0.7299444  0.726096   0.7591712  0.69265276 0.99999994]]


In [ ]:
# def naturalize_code(code):
#     replacements = {
#         'ÃŽÂµ': 'end of string or no character',
#         'Concat': 'join strings',
#         'ConstStr': 'fixed string',
#         'RegexPair': 'pattern matching',
#         'let columnName = "0" in let x = ChooseInput(vs, columnName) in': 'select column "0" from the data',
#         '\\"': 'quotation mark',
#         'RegexPositionRelative': 'find position relative to pattern',
#         'SubStr': 'extract substring',
#         'PosPair': 'position pair indicating start and end'
#     }

#     # Apply all replacements to the code
#     for old, new in replacements.items():
#         code = re.sub(re.escape(old), new, code)

#     return code

# df['pseudo-coded'] = df.iloc[:, 0].apply(naturalize_code)


# df


,program,transformation,lhs,rhs,pseudo-coded
0,"let columnName = ""0"" in let x = ChooseInput(vs...",Computers as Theatre->Computers As Theatre,Computers as Theatre,Computers As Theatre,"select column ""0"" from the data ToSimpleTitleC..."
1,"Concat(let columnName = ""0"" in let x = ChooseI...",Computers As Theatre->Computers as Theatre,Computers As Theatre,Computers as Theatre,"join strings(select column ""0"" from the data e..."
2,"Concat(ConstStr(""\""""), Concat(let columnName =...","Brenda Laurel->""Laurel, Brenda""",Brenda Laurel,"""Laurel, Brenda""","join strings(fixed string(""quotation mark""), j..."
3,"Concat(let columnName = ""0"" in let x = ChooseI...","""Laurel, Brenda""->Brenda Laurel","""Laurel, Brenda""",Brenda Laurel,"join strings(select column ""0"" from the data e..."
4,"Concat(let columnName = ""0"" in let x = ChooseI...",Tru64 Unix Troubleshooting Diagnosing &amp;->T...,Tru64 Unix Troubleshooting Diagnosing &amp;,Tru64 UNIX Troubleshooting : Diagnosing and Co...,"join strings(select column ""0"" from the data e..."
...,...,...,...,...,...
485,"Concat(ConstStr(""\""""), Concat(let columnName =...","""Farin, Gerald""->""Gerald E. Farin,""","""Farin, Gerald""","""Gerald E. Farin,""","join strings(fixed string(""quotation mark""), j..."
486,"Concat(let columnName = ""0"" in let x = ChooseI...",Computer Systems Design and Architecture->Comp...,Computer Systems Design and Architecture,Computer Systems Design and Architecture (2nd ...,"join strings(select column ""0"" from the data x..."
487,"let columnName = ""0"" in let x = ChooseInput(vs...",Computer Systems Design and Architecture (2nd ...,Computer Systems Design and Architecture (2nd ...,Computer Systems Design and Architecture,"select column ""0"" from the data extract substr..."
488,"Concat(ConstStr(""\""Vincent P. ""), Concat(let c...","Heuring->""Vincent P. Heuring, Harry F. Jordan""",Heuring,"""Vincent P. Heuring, Harry F. Jordan""","join strings(fixed string(""quotation markVince..."


In [7]:
transformations = df.iloc[:, 0]
transformation_embeddings = [model.encode(text) for text in transformations]

In [8]:
np.save('embeddings/transformation_embeddings.npy', transformation_embeddings)

In [9]:
cosine_similarities = cosine_similarity(transformation_embeddings)
print(cosine_similarities.shape)

(490, 490)


In [10]:
# Brute force approach: Group together all indices where cosine similarity is > 0.8
# undirected_graph = {} 
# for i in range(len(cosine_similarities)):
#     undirected_graph[i] = []

# for i in range(len(cosine_similarities)):
#     for j in range(i+1, len(cosine_similarities[0])):
#         if cosine_similarities[i][j] > 0.8:
#             undirected_graph[i].append(j)
#             undirected_graph[j].append(i)
simiarity_theshold = 0.98

row_indices, col_indices = np.where(cosine_similarities > simiarity_theshold)

groups = {}
undirected_graph = {} 


# Iterate over unique row indices
for i in np.unique(row_indices):
    relevant_cols = col_indices[row_indices == i]

    if i not in undirected_graph:
        undirected_graph[i] = []
    undirected_graph[i].extend(relevant_cols.tolist())

    for col in relevant_cols:
        if col not in undirected_graph:
            undirected_graph[col] = []
        undirected_graph[col].append(i)
    
    # Store relevant column indices in groups
    groups[i] = relevant_cols.tolist()

# Convert the lists to sets to remove duplicates in the undirected graph
for key in undirected_graph:
    undirected_graph[key] = list(set(undirected_graph[key]))

for i in range(len(df)):
    if i not in groups:
        groups[i] = []
    if i not in undirected_graph:
        undirected_graph[i] = [i]

In [11]:
graph = groups.copy()
clusters = []
visited = set()

def dfs(v, curr_cluster):
    visited.add(v)
    curr_cluster.append(v)
    for w in graph[v]:
        if w not in visited:
            dfs(w, curr_cluster)

for node in graph:
    if node not in visited:
        cluster = []
        dfs(node, cluster)
        clusters.append(cluster)

# print(clusters)

In [12]:
print(clusters)

[[0, 83, 85, 119, 159, 185], [1], [2, 7, 64, 92, 19, 61, 89, 96, 101, 104, 108, 113, 129, 140, 171, 174, 178, 187, 192, 200, 182, 240, 212, 216, 220, 224, 232, 244, 253, 264, 268, 277, 282, 286, 290, 299, 302, 318, 322, 366, 347, 362, 390, 395, 414, 418, 434, 443, 465, 476, 484], [3, 93, 97, 100, 112, 141, 175, 186, 193, 208, 209, 213, 217, 221, 233, 252, 269, 287, 319, 367, 394, 415, 419, 435, 442, 472, 18, 88, 276, 324, 477, 473], [4, 223, 143, 413, 148, 475, 206, 336, 425, 263], [5], [6], [8], [9, 78, 102, 114, 210, 218, 230, 238, 254, 266, 300, 349, 352, 377, 429, 449, 483, 487], [10, 15, 49, 37, 11, 14, 48, 36, 117, 132, 133, 136, 137, 469, 261, 248, 260, 306, 314, 315, 307, 371, 370, 378, 379, 382, 383, 407, 426, 430, 431, 450, 468, 406, 451, 333, 169, 427, 128], [12], [13], [16], [17], [20], [21], [22, 77, 335, 52, 53, 334], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32, 43, 45, 58, 87, 124, 153, 162, 229, 272, 326, 330, 343, 345, 398, 351, 439], [33], [34], [35, 361

In [13]:
with open('../outputs/clusters-PROSE.txt', 'w') as f:
    for clus in clusters:
        f.write('[')
        for idx in clus:
            f.write(f'{transformations.iloc[idx]}; ')
        f.write(']\n\n')

str1Tostr2 = df.iloc[:, 1]
with open('../outputs/str1Tostr2Clusters.txt', 'w') as f:
    for clus in clusters:
        f.write('[')
        for idx in clus:
            f.write(f'{str1Tostr2.iloc[idx]}; ')
        f.write(']\n\n')

In [14]:
from collections import Counter

lengths =[]
for cluster in clusters:
    lengths.append(len(cluster))
print(Counter(lengths))
unique_values_count = df.iloc[:, 0].nunique()
print("Number of unique values in program column:", unique_values_count)

Counter({1: 222, 2: 15, 6: 2, 51: 1, 32: 1, 10: 1, 18: 1, 39: 1, 17: 1, 19: 1, 9: 1, 11: 1, 5: 1, 8: 1, 4: 1, 3: 1})
Number of unique values in program column: 362


Counter({1: 236, 2: 17, 3: 4, 4: 3, 6: 1, 50: 1, 27: 1, 34: 1, 36: 1, 10: 1, 9: 1, 11: 1, 5: 1, 8: 1})

In [16]:
# Read in text file that shows PROSE program to transformation
file_path = '../data/programs_one_to_one_conflict.txt'

with open(file_path, 'r') as file:
    data = file.read()

# Split the data on the word "Program:" but keep it as part of the split results
programs_transformations = data.split("\n")
programs_transformations.remove("")

programs = []
transformations = []
lhs = []
rhs = []

for item in programs_transformations:
    item_split = item.split("==>")
    transformation = item_split[1]
    programs.append(item_split[0])
    transformations.append(transformation)
    transformation_split = transformation.split("->")
    lhs.append(transformation_split[0])
    rhs.append(transformation_split[1])



df = pd.DataFrame({
    'program': programs,
    'transformation': transformations,
    'lhs': lhs,
    'rhs': rhs
})

df

,program,transformation,lhs,rhs
0,"Concat(ConstStr(""Hack ""), Concat(let columnNam...",ADVANCES IN COMPUTERS V56->Hack I.T. - Securit...,ADVANCES IN COMPUTERS V56,Hack I.T. - Security Through Penetration Testi...
1,"Concat(let columnName = ""0"" in let x = ChooseI...",Hack I.T. - Security Through Penetration Testi...,Hack I.T. - Security Through Penetration Testi...,ADVANCES IN COMPUTERS V56
2,"Concat(ConstStr(""\""Klevinsky, T. J.""), let col...","""Zelkowitz, Marvin""->""Klevinsky, T. J.""","""Zelkowitz, Marvin""","""Klevinsky, T. J."""
3,"Concat(ConstStr(""\""Zelkowitz, Marvin""), let co...","""Klevinsky, T. J.""->""Zelkowitz, Marvin""","""Klevinsky, T. J.""","""Zelkowitz, Marvin"""
4,"Concat(ConstStr(""Communication Networking an A...",Wi-Foo: The Secrets of Wireless Hacking->Commu...,Wi-Foo: The Secrets of Wireless Hacking,Communication Networking an Analytical A
...,...,...,...,...
495,"Concat(ConstStr(""\""Beveridge, Jim, Wiener, Rob...","""Fortier,Paul""->""Beveridge, Jim, Wiener, Robert""","""Fortier,Paul""","""Beveridge, Jim, Wiener, Robert"""
496,"Concat(ConstStr(""Accelerated ""), Concat(let co...",Data Structures With C++ Using Stl->Accelerate...,Data Structures With C++ Using Stl,Accelerated C++ Practical Prog By Exampl
497,"Concat(ConstStr(""Data Structures With ""), Conc...",Accelerated C++ Practical Prog By Exampl->Data...,Accelerated C++ Practical Prog By Exampl,Data Structures With C++ Using Stl
498,"Concat(ConstStr(""\""Koenig, Andrew""), let colum...","""Ford, William; Topp, William; Topp, William R...","""Ford, William; Topp, William; Topp, William R.""","""Koenig, Andrew"""


In [ ]:
# def naturalize_code(code):
#     replacements = {
#         'ÃŽÂµ': 'end of string or no character',
#         'Concat': 'join strings',
#         'ConstStr': 'fixed string',
#         'RegexPair': 'pattern matching',
#         'let columnName = "0" in let x = ChooseInput(vs, columnName) in': 'select column "0" from the data',
#         '\\"': 'quotation mark',
#         'RegexPositionRelative': 'find position relative to pattern',
#         'SubStr': 'extract substring',
#         'PosPair': 'position pair indicating start and end'
#     }

#     # Apply all replacements to the code
#     for old, new in replacements.items():
#         code = re.sub(re.escape(old), new, code)

#     return code

# df['pseudo-coded'] = df.iloc[:, 0].apply(naturalize_code)


df

,program,transformation,lhs,rhs,pseudo-coded
0,"Concat(ConstStr(""Hack ""), Concat(let columnNam...",ADVANCES IN COMPUTERS V56->Hack I.T. - Securit...,ADVANCES IN COMPUTERS V56,Hack I.T. - Security Through Penetration Testi...,"join strings(fixed string(""Hack ""), join strin..."
1,"Concat(let columnName = ""0"" in let x = ChooseI...",Hack I.T. - Security Through Penetration Testi...,Hack I.T. - Security Through Penetration Testi...,ADVANCES IN COMPUTERS V56,"join strings(select column ""0"" from the data T..."
2,"Concat(ConstStr(""\""Klevinsky, T. J.""), let col...","""Zelkowitz, Marvin""->""Klevinsky, T. J.""","""Zelkowitz, Marvin""","""Klevinsky, T. J.""","join strings(fixed string(""quotation markKlevi..."
3,"Concat(ConstStr(""\""Zelkowitz, Marvin""), let co...","""Klevinsky, T. J.""->""Zelkowitz, Marvin""","""Klevinsky, T. J.""","""Zelkowitz, Marvin""","join strings(fixed string(""quotation markZelko..."
4,"Concat(ConstStr(""Communication Networking an A...",Wi-Foo: The Secrets of Wireless Hacking->Commu...,Wi-Foo: The Secrets of Wireless Hacking,Communication Networking an Analytical A,"join strings(fixed string(""Communication Netwo..."
...,...,...,...,...,...
495,"Concat(ConstStr(""\""Beveridge, Jim, Wiener, Rob...","""Fortier,Paul""->""Beveridge, Jim, Wiener, Robert""","""Fortier,Paul""","""Beveridge, Jim, Wiener, Robert""","join strings(fixed string(""quotation markBever..."
496,"Concat(ConstStr(""Accelerated ""), Concat(let co...",Data Structures With C++ Using Stl->Accelerate...,Data Structures With C++ Using Stl,Accelerated C++ Practical Prog By Exampl,"join strings(fixed string(""Accelerated ""), joi..."
497,"Concat(ConstStr(""Data Structures With ""), Conc...",Accelerated C++ Practical Prog By Exampl->Data...,Accelerated C++ Practical Prog By Exampl,Data Structures With C++ Using Stl,"join strings(fixed string(""Data Structures Wit..."
498,"Concat(ConstStr(""\""Koenig, Andrew""), let colum...","""Ford, William; Topp, William; Topp, William R...","""Ford, William; Topp, William; Topp, William R.""","""Koenig, Andrew""","join strings(fixed string(""quotation markKoeni..."


In [17]:
transformations = df.iloc[:, 0]
transformation_embeddings = [model.encode(text) for text in transformations]
cosine_similarities = cosine_similarity(transformation_embeddings)
print(cosine_similarities.shape)

(500, 500)


In [18]:
row_indices, col_indices = np.where(cosine_similarities > 0.99)

groups = {}
undirected_graph = {} 


# Iterate over unique row indices
for i in np.unique(row_indices):
    # Find the column indices where cosine_similarities > 0.8 for this row
    relevant_cols = col_indices[row_indices == i]

    if i not in undirected_graph:
        undirected_graph[i] = []
    undirected_graph[i].extend(relevant_cols.tolist())

    for col in relevant_cols:
        if col not in undirected_graph:
            undirected_graph[col] = []
        undirected_graph[col].append(i)
    
    # Store relevant column indices in groups
    groups[i] = relevant_cols.tolist()

# Convert the lists to sets to remove duplicates in the undirected graph
for key in undirected_graph:
    undirected_graph[key] = list(set(undirected_graph[key]))

for i in range(len(df)):
    if i not in groups:
        groups[i] = []
    if i not in undirected_graph:
        undirected_graph[i] = [i]

In [19]:
graph = groups.copy()
clusters = []
visited = set()

def dfs(v, curr_cluster):
    visited.add(v)
    curr_cluster.append(v)
    for w in graph[v]:
        if w not in visited:
            dfs(w, curr_cluster)

for node in graph:
    if node not in visited:
        cluster = []
        dfs(node, cluster)
        clusters.append(cluster)

In [20]:
lengths =[]
for cluster in clusters:
    lengths.append(len(cluster))
print(Counter(lengths))
unique_values_count = df.iloc[:, 0].nunique()
print("Number of unique values in 'isbn' column:", unique_values_count)

Counter({1: 485, 2: 6, 3: 1})
Number of unique values in 'isbn' column: 496


Counter({1: 488, 2: 6})